# Baseline simple

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass

!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package libfuse2:amd64.
(Reading database ... 22280 files and directories currently installed.)
Preparing to unpack .../libfuse2_2.9.7-1ubuntu1_amd64.deb ...
Unpacking libfuse2:amd64 (2.9.7-1ubuntu1) ...
Selecting previously unselected package fuse.
Preparing to unpack .../fuse_2.9.7-1ubuntu1_amd64.deb ...
Unpacking fuse (2.9.7-1ubuntu1) ...
Selecting previously unselected package google-drive-ocamlfuse.
Preparing to unpack .../google-drive-ocamlfuse_0.7.0-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.0-0ubuntu1~ubuntu18.04.1) ...
Setting up libfuse2:amd64 (2.9.7-1ubuntu1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
Setting up fuse (2.9.7-1ubuntu1) ...
Setting up google-drive-ocamlfuse (0.7.0-0ubuntu1~ubuntu18.04.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleus

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
from time import time
import os
import gzip
import numpy as np
from keras.utils import np_utils

import matplotlib.pyplot as plt

In [0]:
def load_mnist(path, kind='train'):
    import os
    import gzip
    import numpy as np

    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels

In [0]:
X_train, y_train = load_mnist('drive/TCC-ITAU/fashion-mnist/data/fashion', kind='train')
X_test, y_test = load_mnist('drive/TCC-ITAU/fashion-mnist/data/fashion', kind='t10k')

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1974)

In [0]:
train_features = X_train.reshape(X_train.shape[0], 28, 28, 1)/255
vali_features = X_val.reshape(X_val.shape[0], 28, 28, 1)/255
test_features = X_test.reshape(X_test.shape[0], 28, 28, 1)/255

y_train_cat = np_utils.to_categorical(y_train)
y_val_cat = np_utils.to_categorical(y_val)
y_test_cat = np_utils.to_categorical(y_test)

In [0]:
import keras
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Input
from keras import optimizers
from keras.callbacks import TensorBoard, ReduceLROnPlateau, CSVLogger
from keras.layers.normalization import BatchNormalization
from keras.utils.data_utils import get_file
from keras import backend as K

In [0]:
img_rows, img_cols = 28, 28
if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

In [0]:
num_classes_fine = 10

img_input = Input(shape=input_shape, name='input')


x = Conv2D(32, (3, 3), activation='relu', name='block1_conv1')(img_input)
x = MaxPooling2D((2, 2), name='block1_pool')(x)

x = Conv2D(64, (3, 3), activation='relu', name='block1_conv2')(x)
x = MaxPooling2D((2, 2), name='block2_pool')(x)



x = Flatten(name='flatten')(x)
x = Dense(128, name='fc_1')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)

fine_pred = Dense(num_classes_fine, activation='softmax', name='predictions_fine')(x)

In [0]:
model = Model(inputs=img_input, outputs= fine_pred, name='baseline_net')

In [39]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 28, 28, 1)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 26, 26, 32)        320       
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 13, 13, 32)        0         
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 11, 11, 64)        18496     
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
fc_1 (Dense)                 (None, 128)               204928    
__________

In [0]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
class TimingCallback(keras.callbacks.Callback):
    
  def on_epoch_begin(self, epoch, logs={}):
    self.starttime=time()
  def on_epoch_end(self, epoch, logs={}):
    logs['time_elapsed'] = (time()-self.starttime)

In [0]:
tb_cb = TensorBoard(log_dir='drive/TCC-ITAU/fashion-mnist/tensorboard/baseline-simple', histogram_freq=0)
clr_cb = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=1)
time_cb = TimingCallback()
csv_cb = CSVLogger('drive/TCC-ITAU/fashion-mnist/training-data/baseline/baseline-simple.csv', separator=',', append=False)

cbks = [tb_cb, clr_cb, time_cb, csv_cb]

In [43]:
epochs = 50
start_time = time()
training = model.fit(train_features, y_train_cat,
                     validation_data=(vali_features, y_val_cat),
                     epochs=epochs,
                     batch_size=512, 
                     callbacks=cbks,
                     verbose=1)
training_time = time() - start_time

Train on 48000 samples, validate on 12000 samples
Epoch 1/50
48000/48000 [==============================] - 3s 64us/step - loss: 0.6480 - acc: 0.7727 - val_loss: 0.4163 - val_acc: 0.8546
Epoch 2/50
48000/48000 [==============================] - 3s 54us/step - loss: 0.3851 - acc: 0.8655 - val_loss: 0.4586 - val_acc: 0.8110
Epoch 3/50
48000/48000 [==============================] - 3s 53us/step - loss: 0.3230 - acc: 0.8863 - val_loss: 0.3023 - val_acc: 0.8883
Epoch 4/50
48000/48000 [==============================] - 3s 54us/step - loss: 0.2899 - acc: 0.8961 - val_loss: 0.3063 - val_acc: 0.8853
Epoch 5/50
48000/48000 [==============================] - 3s 53us/step - loss: 0.2687 - acc: 0.9046 - val_loss: 0.2905 - val_acc: 0.8921
Epoch 6/50
48000/48000 [==============================] - 3s 53us/step - loss: 0.2460 - acc: 0.9129 - val_loss: 0.2659 - val_acc: 0.9024
Epoch 7/50
48000/48000 [==============================] - 3s 53us/step - loss: 0.2296 - acc: 0.9178 - val_loss: 0.2656 - val_acc

In [44]:
model.evaluate(test_features, y_test_cat)

10000/10000 [==============================] - 1s 100us/step


[0.37469776233434676, 0.9118]

In [0]:
model.save('drive/TCC-ITAU/fashion-mnist/models/baseline-simple.h5')

In [46]:
training_time

135.72356009483337